<a href="https://www.kaggle.com/code/nigamshitij/extract-amazon-ratings?scriptVersionId=193705892" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-review-samples/goodreads_fantasy.csv
/kaggle/input/book-review-samples/goodreads_all_genres.csv
/kaggle/input/book-review-samples/Darth_Plagueis_reviews(4).csv
/kaggle/input/book-review-samples/Dune_Book_1_reviews.csv
/kaggle/input/book-review-samples/Dune_Book_1_reviews(4).csv


In [18]:
import csv
import requests
from bs4 import BeautifulSoup
import time
import random
import os
import re

In [26]:
def get_amazon_link(goodreads_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # Convert the main book URL to the "get a copy" URL
        book_id = re.search(r'/show/(\d+)', goodreads_url).group(1)
        get_copy_url = f"https://www.goodreads.com/book/{book_id}/get_a_copy"
        
        print(get_copy_url)
        
        response = requests.get(get_copy_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the Amazon link
        amazon_link = soup.find('a', href=lambda href: href and 'amazon.com' in href)
        
        if amazon_link:
            return amazon_link['href']
        else:
            print(f"No Amazon link found for {goodreads_url}")
            return None
    
    except Exception as e:
        print(f"Error fetching Amazon link for {goodreads_url}: {str(e)}")
        return None

In [27]:
def get_amazon_ratings(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the star rating
        star_rating = soup.find('span', {'class': 'a-icon-alt'})
        if star_rating:
            star_rating = star_rating.text.split(' out of ')[0]
        else:
            star_rating = "N/A"
        
        # Find the total number of ratings
        total_ratings = soup.find('span', {'id': 'acrCustomerReviewText'})
        if total_ratings:
            total_ratings = total_ratings.text.split(' ratings')[0].replace(',', '')
        else:
            total_ratings = "N/A"
        
        return star_rating, total_ratings
    
    except Exception as e:
        print(f"Error fetching data for {url}: {str(e)}")
        return "N/A", "N/A"

In [28]:
def remove_duplicates(input_file, output_file):
    seen = set()
    with open(input_file, 'r', newline='', encoding='utf-8') as infile, \
         open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        
        for row in reader:
            # Create a tuple of values that define uniqueness
            key = (row['Title'], row['Authors'])
            if key not in seen:
                seen.add(key)
                writer.writerow(row)

In [29]:
def process_csv(input_file, output_file):
    # First, remove duplicates
    temp_file = 'temp_no_duplicates.csv'
    remove_duplicates(input_file, temp_file)
    
    with open(temp_file, 'r', newline='', encoding='utf-8') as infile, \
         open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['Amazon URL', 'Amazon Star Rating', 'Amazon Total Ratings']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for row in reader:
            goodreads_url = row['URL']
            amazon_url = get_amazon_link(goodreads_url)
            row['Amazon URL'] = amazon_url if amazon_url else "N/A"
            
            if amazon_url:
                star_rating, total_ratings = get_amazon_ratings(amazon_url)
                row['Amazon Star Rating'] = star_rating
                row['Amazon Total Ratings'] = total_ratings
            else:
                row['Amazon Star Rating'] = "N/A"
                row['Amazon Total Ratings'] = "N/A"
            
            writer.writerow(row)
            
            # Add a delay to avoid overwhelming the servers
            time.sleep(random.uniform(2, 4))
    
    # Clean up the temporary file
    os.remove(temp_file)

In [30]:
path = '/kaggle/input/book-review-samples/goodreads_all_genres.csv'

In [31]:
if __name__ == "__main__":
    input_file = path
    output_file = 'output_books_with_amazon_ratings.csv'
    process_csv(input_file, output_file)
    print("Processing complete. Check the output file.")

https://www.goodreads.com/book/625603/get_a_copy
No Amazon link found for https://www.goodreads.com/book/show/625603.Horus_Rising
https://www.goodreads.com/book/381817/get_a_copy
No Amazon link found for https://www.goodreads.com/book/show/381817.False_Gods
https://www.goodreads.com/book/815091/get_a_copy


KeyboardInterrupt: 